In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data.dataset import random_split
from tensorboardX import SummaryWriter
from torch.utils.data.sampler import  WeightedRandomSampler

import os

In [2]:
'''
https://github.com/KellerJordan/ResNet-PyTorch-CIFAR10
'''

class ResNet(nn.Module):
    
    def __init__(self, n=7, res_option='A', use_dropout=False):
        super(ResNet, self).__init__()
        self.res_option = res_option
        self.use_dropout = use_dropout
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.norm1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU(inplace=True)
        self.layers1 = self._make_layer(n, 16, 16, 1)
        self.layers2 = self._make_layer(n, 32, 16, 2)
        self.layers3 = self._make_layer(n, 64, 32, 2)
        self.avgpool = nn.AvgPool2d(8)
        self.linear = nn.Linear(64, 10)
    
    def _make_layer(self, layer_count, channels, channels_in, stride):
        return nn.Sequential(
            ResBlock(channels, channels_in, stride, res_option=self.res_option, use_dropout=self.use_dropout),
            *[ResBlock(channels) for _ in range(layer_count-1)])
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu1(out)
        out = self.layers1(out)
        out = self.layers2(out)
        out = self.layers3(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

class ResBlock(nn.Module):
    
    def __init__(self, num_filters, channels_in=None, stride=1, res_option='A', use_dropout=False):
        super(ResBlock, self).__init__()
        
        # uses 1x1 convolutions for downsampling
        if not channels_in or channels_in == num_filters:
            channels_in = num_filters
            self.projection = None
        else:
            if res_option == 'A':
                self.projection = IdentityPadding(num_filters, channels_in, stride)
            elif res_option == 'B':
                self.projection = ConvProjection(num_filters, channels_in, stride)
            elif res_option == 'C':
                self.projection = AvgPoolPadding(num_filters, channels_in, stride)
        self.use_dropout = use_dropout

        self.conv1 = nn.Conv2d(channels_in, num_filters, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(num_filters)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(num_filters, num_filters, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_filters)
        if self.use_dropout:
            self.dropout = nn.Dropout(inplace=True)
        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.use_dropout:
            out = self.dropout(out)
        if self.projection:
            residual = self.projection(x)
        out += residual
        out = self.relu2(out)
        return out


# various projection options to change number of filters in residual connection
# option A from paper
class IdentityPadding(nn.Module):
    def __init__(self, num_filters, channels_in, stride):
        super(IdentityPadding, self).__init__()
        # with kernel_size=1, max pooling is equivalent to identity mapping with stride
        self.identity = nn.MaxPool2d(1, stride=stride)
        self.num_zeros = num_filters - channels_in
    
    def forward(self, x):
        out = F.pad(x, (0, 0, 0, 0, 0, self.num_zeros))
        out = self.identity(out)
        return out

# option B from paper
class ConvProjection(nn.Module):

    def __init__(self, num_filters, channels_in, stride):
        super(ResA, self).__init__()
        self.conv = nn.Conv2d(channels_in, num_filters, kernel_size=1, stride=stride)
    
    def forward(self, x):
        out = self.conv(x)
        return out

# experimental option C
class AvgPoolPadding(nn.Module):

    def __init__(self, num_filters, channels_in, stride):
        super(AvgPoolPadding, self).__init__()
        self.identity = nn.AvgPool2d(stride, stride=stride)
        self.num_zeros = num_filters - channels_in
    
    def forward(self, x):
        out = F.pad(x, (0, 0, 0, 0, 0, self.num_zeros))
        out = self.identity(out)
        return out

Preperations for LID sampling.

In [3]:
lid_file = './tmp/train_lid_k100'
dataroot = 'd:/Lab/dataset'
lid_res = torch.load(lid_file)
lid_vec = np.mean(lid_res, axis=1)
lid_idx = np.argsort(lid_vec)

train_dataset = torchvision.datasets.CIFAR10(root=dataroot, train=True, download=False, transform=transforms.ToTensor())
train_size = len(train_dataset)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

dist = [[] for _ in range(10)]
label_to_idx = [[] for _ in range(10)]
for m in range(train_size):
    _, label = train_dataset[m]
    dist[label].append(lid_vec[m])
    label_to_idx[label].append(m)    
    
def _softmax(p, mask=None):
    ep = np.exp(p)
    if not mask is None:
        ep *= mask
    ep /= np.sum(ep)
    return ep

In [4]:
from datetime import datetime
now = datetime.now()
time_suffix = 'LID_' + now.strftime("%Y%m%d-%H%M%S")

dataroot = 'd:/lab/dataset/'
b_size = 200
n_epochs = 10
lr = 0.01
best_loss = 100000
n_iter = 0
test_step = 100
model_file = './tmp/model1_%s.md' % time_suffix

device = 'cuda' if torch.cuda.is_available() else 'cpu'
writer_train = SummaryWriter('./tmp/log_train_%s' % time_suffix)
writer_val = SummaryWriter('./tmp/log_test_%s' % time_suffix)

def test_validate(model, device, test_loader, test_valid='Test'):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss(reduction='sum')(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    writer_val.add_scalar('Loss', test_loss, n_iter)
    writer_val.add_scalar('Acc', acc, n_iter)
    print('\n{} set: Average loss: {:.6f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
        test_valid, test_loss, correct, len(test_loader.dataset), acc))
    return test_loss
    
def train(model, device, train_loader_per_class, test_loader, optimizer, epoch):
    global best_loss, patience, n_iter
    batch_idx = 0
    for _ in range(train_size // b_size):
        loader_iters = map(iter, train_loader_per_class)
        datas, targets = [], []
        for train_loader in loader_iters:
            data, target = train_loader.next()
            datas.append(data)
            targets.append(target)
        #print(datas, targets)
        data, target = torch.cat(datas), torch.cat(targets)
        data, target = data.to(device), target.to(device)
        model.train()
        batch_idx += 1
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct = pred.eq(target.view_as(pred)).sum().item()
        acc = 100. * correct / b_size
        writer_train.add_scalar('Loss', loss.item(), n_iter)
        writer_train.add_scalar('Acc', acc, n_iter)
        n_iter += 1
        optimizer.step()
        if batch_idx % 1 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx * b_size / train_size, loss.item()))
        if n_iter % test_step == 0:
            print('Start testing...')
            test_loss = test_validate(model, device, test_loader)

def main():        
    dataroot = 'D:\Lab\dataset'
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    train_dataset = torchvision.datasets.CIFAR10(root=dataroot, train=True, download=False, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root=dataroot, train=False, download=False, transform=transform_test)
    
    train_size = len(train_dataset)
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=b_size, shuffle=False, num_workers=2)

    model = ResNet(9, res_option='A', use_dropout=False).to(device)
    print(model)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)

    temp_list = [0.05, 0.1, 0.5, 1, 2, 5, 10]
    for epoch in range(1, n_epochs + 1):
        if epoch - 1 < len(temp_list):
            T = temp_list[epoch - 1]
        else:
            T = temp_list[-1]
        print('Set temperature: %f' % T)
        train_loader_per_class = []
        for c in range(10):
            mask = np.zeros(train_size)
            mask[np.array(label_to_idx[c])] = 1
            a_vec = _softmax(-lid_vec / T, mask)
            sampler = WeightedRandomSampler(a_vec,\
                                            num_samples=b_size // 10,\
                                            replacement=False)
            train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=b_size // 10, shuffle=False,
                                                    sampler=sampler)
            train_loader_per_class.append(train_loader)
        train(model, device, train_loader_per_class, test_loader, optimizer, epoch)          
        
main()
writer_train.close()
writer_val.close()

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace)
  (layers1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace)
    )
    (1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, m

Train Epoch: 1 [200/50000 (0%)]	Loss: 2.495326
Train Epoch: 1 [400/50000 (1%)]	Loss: 2.875619
Train Epoch: 1 [600/50000 (1%)]	Loss: 2.440843
Train Epoch: 1 [800/50000 (2%)]	Loss: 2.062562
Train Epoch: 1 [1000/50000 (2%)]	Loss: 2.105644
Train Epoch: 1 [1200/50000 (2%)]	Loss: 1.992285
Train Epoch: 1 [1400/50000 (3%)]	Loss: 1.903150
Train Epoch: 1 [1600/50000 (3%)]	Loss: 1.935723
Train Epoch: 1 [1800/50000 (4%)]	Loss: 1.792383
Train Epoch: 1 [2000/50000 (4%)]	Loss: 1.736482
Train Epoch: 1 [2200/50000 (4%)]	Loss: 1.699358
Train Epoch: 1 [2400/50000 (5%)]	Loss: 1.605003
Train Epoch: 1 [2600/50000 (5%)]	Loss: 1.525826
Train Epoch: 1 [2800/50000 (6%)]	Loss: 1.513818
Train Epoch: 1 [3000/50000 (6%)]	Loss: 1.414329
Train Epoch: 1 [3200/50000 (6%)]	Loss: 1.332814
Train Epoch: 1 [3400/50000 (7%)]	Loss: 1.325020
Train Epoch: 1 [3600/50000 (7%)]	Loss: 1.296694
Train Epoch: 1 [3800/50000 (8%)]	Loss: 1.231378
Train Epoch: 1 [4000/50000 (8%)]	Loss: 1.207221
Train Epoch: 1 [4200/50000 (8%)]	Loss: 1.154

Train Epoch: 1 [33000/50000 (66%)]	Loss: 0.209359
Train Epoch: 1 [33200/50000 (66%)]	Loss: 0.207034
Train Epoch: 1 [33400/50000 (67%)]	Loss: 0.203118
Train Epoch: 1 [33600/50000 (67%)]	Loss: 0.182603
Train Epoch: 1 [33800/50000 (68%)]	Loss: 0.128677
Train Epoch: 1 [34000/50000 (68%)]	Loss: 0.237274
Train Epoch: 1 [34200/50000 (68%)]	Loss: 0.150704
Train Epoch: 1 [34400/50000 (69%)]	Loss: 0.131017
Train Epoch: 1 [34600/50000 (69%)]	Loss: 0.196551
Train Epoch: 1 [34800/50000 (70%)]	Loss: 0.195411
Train Epoch: 1 [35000/50000 (70%)]	Loss: 0.155051
Train Epoch: 1 [35200/50000 (70%)]	Loss: 0.111927
Train Epoch: 1 [35400/50000 (71%)]	Loss: 0.123708
Train Epoch: 1 [35600/50000 (71%)]	Loss: 0.094394
Train Epoch: 1 [35800/50000 (72%)]	Loss: 0.129155
Train Epoch: 1 [36000/50000 (72%)]	Loss: 0.095988
Train Epoch: 1 [36200/50000 (72%)]	Loss: 0.138783
Train Epoch: 1 [36400/50000 (73%)]	Loss: 0.114642
Train Epoch: 1 [36600/50000 (73%)]	Loss: 0.119628
Train Epoch: 1 [36800/50000 (74%)]	Loss: 0.064316


Train Epoch: 2 [15400/50000 (31%)]	Loss: 0.091563
Train Epoch: 2 [15600/50000 (31%)]	Loss: 0.083507
Train Epoch: 2 [15800/50000 (32%)]	Loss: 0.130034
Train Epoch: 2 [16000/50000 (32%)]	Loss: 0.193240
Train Epoch: 2 [16200/50000 (32%)]	Loss: 0.151264
Train Epoch: 2 [16400/50000 (33%)]	Loss: 0.088552
Train Epoch: 2 [16600/50000 (33%)]	Loss: 0.053098
Train Epoch: 2 [16800/50000 (34%)]	Loss: 0.151943
Train Epoch: 2 [17000/50000 (34%)]	Loss: 0.087259
Train Epoch: 2 [17200/50000 (34%)]	Loss: 0.160928
Train Epoch: 2 [17400/50000 (35%)]	Loss: 0.133740
Train Epoch: 2 [17600/50000 (35%)]	Loss: 0.144548
Train Epoch: 2 [17800/50000 (36%)]	Loss: 0.079174
Train Epoch: 2 [18000/50000 (36%)]	Loss: 0.147396
Train Epoch: 2 [18200/50000 (36%)]	Loss: 0.049067
Train Epoch: 2 [18400/50000 (37%)]	Loss: 0.127168
Train Epoch: 2 [18600/50000 (37%)]	Loss: 0.125437
Train Epoch: 2 [18800/50000 (38%)]	Loss: 0.176951
Train Epoch: 2 [19000/50000 (38%)]	Loss: 0.125658
Train Epoch: 2 [19200/50000 (38%)]	Loss: 0.085162


Train Epoch: 2 [48000/50000 (96%)]	Loss: 0.116546
Train Epoch: 2 [48200/50000 (96%)]	Loss: 0.153756
Train Epoch: 2 [48400/50000 (97%)]	Loss: 0.161554
Train Epoch: 2 [48600/50000 (97%)]	Loss: 0.140192
Train Epoch: 2 [48800/50000 (98%)]	Loss: 0.223767
Train Epoch: 2 [49000/50000 (98%)]	Loss: 0.170851
Train Epoch: 2 [49200/50000 (98%)]	Loss: 0.076507
Train Epoch: 2 [49400/50000 (99%)]	Loss: 0.145756
Train Epoch: 2 [49600/50000 (99%)]	Loss: 0.113380
Train Epoch: 2 [49800/50000 (100%)]	Loss: 0.147697
Train Epoch: 2 [50000/50000 (100%)]	Loss: 0.114359
Start testing...

Test set: Average loss: 5.684921, Accuracy: 2982/10000 (29.820000%)

Set temperature: 0.500000
Train Epoch: 3 [200/50000 (0%)]	Loss: 1.141431
Train Epoch: 3 [400/50000 (1%)]	Loss: 1.470171
Train Epoch: 3 [600/50000 (1%)]	Loss: 1.243797
Train Epoch: 3 [800/50000 (2%)]	Loss: 1.623118
Train Epoch: 3 [1000/50000 (2%)]	Loss: 1.249347
Train Epoch: 3 [1200/50000 (2%)]	Loss: 1.316634
Train Epoch: 3 [1400/50000 (3%)]	Loss: 1.081604
Tra

...

Train Epoch: 8 [7800/50000 (16%)]	Loss: 0.724585
Train Epoch: 8 [8000/50000 (16%)]	Loss: 0.690523
Train Epoch: 8 [8200/50000 (16%)]	Loss: 0.646419
Train Epoch: 8 [8400/50000 (17%)]	Loss: 0.758731
Train Epoch: 8 [8600/50000 (17%)]	Loss: 0.712389
Train Epoch: 8 [8800/50000 (18%)]	Loss: 0.654622
Train Epoch: 8 [9000/50000 (18%)]	Loss: 0.637378
Train Epoch: 8 [9200/50000 (18%)]	Loss: 0.582878
Train Epoch: 8 [9400/50000 (19%)]	Loss: 0.590114
Train Epoch: 8 [9600/50000 (19%)]	Loss: 0.655494
Train Epoch: 8 [9800/50000 (20%)]	Loss: 0.683883
Train Epoch: 8 [10000/50000 (20%)]	Loss: 0.597443
Start testing...

Test set: Average loss: 1.092562, Accuracy: 6570/10000 (65.700000%)

Train Epoch: 8 [10200/50000 (20%)]	Loss: 0.692646
Train Epoch: 8 [10400/50000 (21%)]	Loss: 0.679163
Train Epoch: 8 [10600/50000 (21%)]	Loss: 0.628763
Train Epoch: 8 [10800/50000 (22%)]	Loss: 0.535647
Train Epoch: 8 [11000/50000 (22%)]	Loss: 0.701608
Train Epoch: 8 [11200/50000 (22%)]	Loss: 0.717977
Train Epoch: 8 [11400/50

Train Epoch: 8 [40000/50000 (80%)]	Loss: 0.531566
Train Epoch: 8 [40200/50000 (80%)]	Loss: 0.677022
Train Epoch: 8 [40400/50000 (81%)]	Loss: 0.597993
Train Epoch: 8 [40600/50000 (81%)]	Loss: 0.723627
Train Epoch: 8 [40800/50000 (82%)]	Loss: 0.627592
Train Epoch: 8 [41000/50000 (82%)]	Loss: 0.733789
Train Epoch: 8 [41200/50000 (82%)]	Loss: 0.578498
Train Epoch: 8 [41400/50000 (83%)]	Loss: 0.619462
Train Epoch: 8 [41600/50000 (83%)]	Loss: 0.675792
Train Epoch: 8 [41800/50000 (84%)]	Loss: 0.691731
Train Epoch: 8 [42000/50000 (84%)]	Loss: 0.685929
Train Epoch: 8 [42200/50000 (84%)]	Loss: 0.654943
Train Epoch: 8 [42400/50000 (85%)]	Loss: 0.627456
Train Epoch: 8 [42600/50000 (85%)]	Loss: 0.603886
Train Epoch: 8 [42800/50000 (86%)]	Loss: 0.704975
Train Epoch: 8 [43000/50000 (86%)]	Loss: 0.642547
Train Epoch: 8 [43200/50000 (86%)]	Loss: 0.688749
Train Epoch: 8 [43400/50000 (87%)]	Loss: 0.608610
Train Epoch: 8 [43600/50000 (87%)]	Loss: 0.696327
Train Epoch: 8 [43800/50000 (88%)]	Loss: 0.591626


Train Epoch: 9 [22400/50000 (45%)]	Loss: 0.428450
Train Epoch: 9 [22600/50000 (45%)]	Loss: 0.606930
Train Epoch: 9 [22800/50000 (46%)]	Loss: 0.536165
Train Epoch: 9 [23000/50000 (46%)]	Loss: 0.606840
Train Epoch: 9 [23200/50000 (46%)]	Loss: 0.723793
Train Epoch: 9 [23400/50000 (47%)]	Loss: 0.630317
Train Epoch: 9 [23600/50000 (47%)]	Loss: 0.536804
Train Epoch: 9 [23800/50000 (48%)]	Loss: 0.544583
Train Epoch: 9 [24000/50000 (48%)]	Loss: 0.543866
Train Epoch: 9 [24200/50000 (48%)]	Loss: 0.604973
Train Epoch: 9 [24400/50000 (49%)]	Loss: 0.569353
Train Epoch: 9 [24600/50000 (49%)]	Loss: 0.538035
Train Epoch: 9 [24800/50000 (50%)]	Loss: 0.699925
Train Epoch: 9 [25000/50000 (50%)]	Loss: 0.647023
Train Epoch: 9 [25200/50000 (50%)]	Loss: 0.732266
Train Epoch: 9 [25400/50000 (51%)]	Loss: 0.692595
Train Epoch: 9 [25600/50000 (51%)]	Loss: 0.482599
Train Epoch: 9 [25800/50000 (52%)]	Loss: 0.715921
Train Epoch: 9 [26000/50000 (52%)]	Loss: 0.712211
Train Epoch: 9 [26200/50000 (52%)]	Loss: 0.793652


Train Epoch: 10 [5000/50000 (10%)]	Loss: 0.581657
Train Epoch: 10 [5200/50000 (10%)]	Loss: 0.614433
Train Epoch: 10 [5400/50000 (11%)]	Loss: 0.602260
Train Epoch: 10 [5600/50000 (11%)]	Loss: 0.583667
Train Epoch: 10 [5800/50000 (12%)]	Loss: 0.512675
Train Epoch: 10 [6000/50000 (12%)]	Loss: 0.567991
Train Epoch: 10 [6200/50000 (12%)]	Loss: 0.652483
Train Epoch: 10 [6400/50000 (13%)]	Loss: 0.510555
Train Epoch: 10 [6600/50000 (13%)]	Loss: 0.676664
Train Epoch: 10 [6800/50000 (14%)]	Loss: 0.607771
Train Epoch: 10 [7000/50000 (14%)]	Loss: 0.498890
Train Epoch: 10 [7200/50000 (14%)]	Loss: 0.543916
Train Epoch: 10 [7400/50000 (15%)]	Loss: 0.655891
Train Epoch: 10 [7600/50000 (15%)]	Loss: 0.747054
Train Epoch: 10 [7800/50000 (16%)]	Loss: 0.498817
Train Epoch: 10 [8000/50000 (16%)]	Loss: 0.533485
Train Epoch: 10 [8200/50000 (16%)]	Loss: 0.486499
Train Epoch: 10 [8400/50000 (17%)]	Loss: 0.694699
Train Epoch: 10 [8600/50000 (17%)]	Loss: 0.548016
Train Epoch: 10 [8800/50000 (18%)]	Loss: 0.570722


Train Epoch: 10 [36600/50000 (73%)]	Loss: 0.650152
Train Epoch: 10 [36800/50000 (74%)]	Loss: 0.671953
Train Epoch: 10 [37000/50000 (74%)]	Loss: 0.653186
Train Epoch: 10 [37200/50000 (74%)]	Loss: 0.585753
Train Epoch: 10 [37400/50000 (75%)]	Loss: 0.720927
Train Epoch: 10 [37600/50000 (75%)]	Loss: 0.776334
Train Epoch: 10 [37800/50000 (76%)]	Loss: 0.577167
Train Epoch: 10 [38000/50000 (76%)]	Loss: 0.567001
Train Epoch: 10 [38200/50000 (76%)]	Loss: 0.517634
Train Epoch: 10 [38400/50000 (77%)]	Loss: 0.624211
Train Epoch: 10 [38600/50000 (77%)]	Loss: 0.510678
Train Epoch: 10 [38800/50000 (78%)]	Loss: 0.527849
Train Epoch: 10 [39000/50000 (78%)]	Loss: 0.626457
Train Epoch: 10 [39200/50000 (78%)]	Loss: 0.511969
Train Epoch: 10 [39400/50000 (79%)]	Loss: 0.791548
Train Epoch: 10 [39600/50000 (79%)]	Loss: 0.536034
Train Epoch: 10 [39800/50000 (80%)]	Loss: 0.543842
Train Epoch: 10 [40000/50000 (80%)]	Loss: 0.590808
Train Epoch: 10 [40200/50000 (80%)]	Loss: 0.573452
Train Epoch: 10 [40400/50000 (8